In [1]:
# CDS API
import cdsapi

# Libraries for working with multi-dimensional arrays
import xarray as xr
import pandas as pd
import numpy as np

# Forecast verification metrics with xarray
import xskillscore as xs

# Date and calendar libraries
from dateutil.relativedelta import relativedelta
import calendar

# Libraries for plotting and geospatial data visualisation
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Disable warnings for data download via API and matplotlib (do I need both???)
import warnings
warnings.filterwarnings('ignore')

In [2]:
DATAIN ="/home/mohamed/EHTPIII/MODELISATION/input/DATASETS"
DATAOUT="/home/mohamed/EHTPIII/MODELISATION/output"
config = dict(
    list_vars = ['2m_temperature', ],
    hcstarty = 1993,
    hcendy = 2016,
    start_month = 11,
)
SCOREDIR = DATAOUT + '/SF/proba_scores'

In [3]:
import calendar
import ipywidgets as widgets

# Common labels to be used in plot titles

VARNAMES = {
    't2m' : '2-metre temperature'
}
details="_1993-2016_monthly_mean_11_234_45_-30_-2.5_60"

available_files=["ukmo_602","meteo_france_8","ecmwf_51","eccc_3","eccc_2","dwd_21","cmcc_35"]

CATNAMES=['lower tercile', 'middle tercile', 'upper tercile']

# Change titles font size
plt.rc('axes', titlesize=20)

## *B-Ranked Probability Score (RPS)*

In [4]:
def load_data(file,aggr,metric):
    rps = xr.open_dataset(f'{SCOREDIR}/{file}{details}.{aggr}.{metric}.nc')
    return rps

# Define the plot function
def plot_corr(file,aggr, fcmonth):
    origin_labels = {'institution': f'{file}',
                    'SCORE':"RPS"}
    
    tit_line1 = '{institution} : {SCORE} '.format(**origin_labels)
    tit_line2_base = f'Start month: {calendar.month_abbr[config["start_month"]].upper()}'
    
    if aggr=='1m':
        validmonth = config['start_month'] + (fcmonth-1)
        validmonth = validmonth if validmonth<=12 else validmonth-12
    #     tit_line2 = tit_line2_base + rf' - Valid month: $\bf{calendar.month_abbr[validmonth].upper()}$'
        tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
    elif aggr=='3m':
        validmonths = [vm if vm<=12 else vm-12 for vm in [config['start_month'] + (fcmonth-1) - shift for shift in range(3)]]
        validmonths = [calendar.month_abbr[vm][0] for vm in reversed(validmonths)]
    #     tit_line2 = tit_line2_base + rf' - Valid months: $\bf{"".join(validmonths)}$'
        tit_line2 = tit_line2_base + f' - Valid months: {"".join(validmonths)}'
    else:
        raise BaseException(f'Unexpected aggregation {aggr}')
    rps = load_data(file,aggr,"rps")
    
    # Select data for the chosen forecast month
    thisrps = rps.sel(forecastMonth=fcmonth)
    for var in thisrps.data_vars:
        fig = plt.figure(figsize=(18,10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
        avalues = thisrps[var].values
        cs = plt.contourf(thisrps[var].lon,thisrps[var].lat,avalues,levels=np.linspace(0.,0.5,11),cmap='YlGn_r', extend='max')
        cs.cmap.set_under('purple')
        cb = plt.colorbar(shrink=0.5)
        cb.ax.set_ylabel('RPS',fontsize=12)
        plt.title(tit_line1 + f' {VARNAMES[var]}' + ' (tercile categories)\n' + tit_line2,loc='left')
        plt.tight_layout()  
        plt.show()


# Create interactive widgets for `aggr` and `fcmonth`
aggr_dropdown = widgets.Dropdown(options=['1m', '3m'], value='1m', description='Aggregation:')
fcmonth_slider = widgets.IntSlider(value=3, min=2, max=4, step=1, description='Forecast Month:')
file_droptown=widgets.Dropdown(options=available_files,value="cmcc_35",description="Center")
widgets.interactive(plot_corr, file=file_droptown,aggr=aggr_dropdown, fcmonth=fcmonth_slider)

interactive(children=(Dropdown(description='Center', index=6, options=('ukmo_602', 'meteo_france_8', 'ecmwf_51…

## *C. Area under Relative Operating Characteristic (ROC) curve*

In [5]:
# READ scores xr.Datasets
def plot_roc(file,aggr,fcmonth,icat):
    origin_labels = {'institution': f'{file}',
                    'SCORE':""}
    
    tit_line1 = '{institution} : {SCORE} '.format(**origin_labels)
    tit_line2_base = f'Start month: {calendar.month_abbr[config["start_month"]].upper()}'
    
    if aggr=='1m':
        validmonth = config['start_month'] + (fcmonth-1)
        validmonth = validmonth if validmonth<=12 else validmonth-12
    #     tit_line2 = tit_line2_base + rf' - Valid month: $\bf{calendar.month_abbr[validmonth].upper()}$'
        tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
    elif aggr=='3m':
        validmonths = [vm if vm<=12 else vm-12 for vm in [config['start_month'] + (fcmonth-1) - shift for shift in range(3)]]
        validmonths = [calendar.month_abbr[vm][0] for vm in reversed(validmonths)]
    #     tit_line2 = tit_line2_base + rf' - Valid months: $\bf{"".join(validmonths)}$'
        tit_line2 = tit_line2_base + f' - Valid months: {"".join(validmonths)}'
    else:
        raise BaseException(f'Unexpected aggregation {aggr}')
    
    roc=load_data(file,aggr,"roc")
    thisroc = roc.sel(forecastMonth=fcmonth)
    for var in thisroc.data_vars:
        fig = plt.figure(figsize=(18,10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
        avalues = thisroc.sel(category=icat)[var].values
        cs = plt.contourf(thisroc[var].lon,thisroc[var].lat,avalues,levels=np.linspace(np.nanmin(avalues), np.nanmax(avalues), 20),cmap='YlGn', extend='min')
        cs.cmap.set_under('lightgray')
        cb = plt.colorbar(shrink=0.5)
        cb.ax.set_ylabel('Area under ROC curve',fontsize=12)
        plt.title(tit_line1 + f' {VARNAMES[var]}' + f' ({CATNAMES[icat]})\n' + tit_line2, loc='left')
        plt.tight_layout()  
        plt.show()

file_droptown=widgets.Dropdown(options=available_files,value="cmcc_35",description="Center")
cat_slider=widgets.IntSlider(value=3,min=0,max=2,step=1,description="cat")
fcmonth_slider=widgets.IntSlider(value=3,min=2,max=4,step=1,description="fcmonth")
aggr_droptown=widgets.Dropdown(options=["1m","3m"],value="1m",description="aggr")
widgets.interactive(plot_roc,file=file_droptown,aggr=aggr_droptown,fcmonth=fcmonth_slider,icat=cat_slider)

interactive(children=(Dropdown(description='Center', index=6, options=('ukmo_602', 'meteo_france_8', 'ecmwf_51…

## *D-Area under Relative Operating Characteristic Skill Score (ROCSS) curve**

In [6]:
def plot_roc(file,aggr,fcmonth,icat):
    origin_labels = {'institution': f'{file}',
                    'SCORE':'ROCSS'}
    
    tit_line1 = '{institution} : {SCORE} '.format(**origin_labels)
    tit_line2_base = f'Start month: {calendar.month_abbr[config["start_month"]].upper()}'
    
    if aggr=='1m':
        validmonth = config['start_month'] + (fcmonth-1)
        validmonth = validmonth if validmonth<=12 else validmonth-12
    #     tit_line2 = tit_line2_base + rf' - Valid month: $\bf{calendar.month_abbr[validmonth].upper()}$'
        tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
    elif aggr=='3m':
        validmonths = [vm if vm<=12 else vm-12 for vm in [config['start_month'] + (fcmonth-1) - shift for shift in range(3)]]
        validmonths = [calendar.month_abbr[vm][0] for vm in reversed(validmonths)]
    #     tit_line2 = tit_line2_base + rf' - Valid months: $\bf{"".join(validmonths)}$'
        tit_line2 = tit_line2_base + f' - Valid months: {"".join(validmonths)}'
    else:
        raise BaseException(f'Unexpected aggregation {aggr}')
    
    roc=load_data(file,aggr,"rocss")
    thisroc = roc.sel(forecastMonth=fcmonth)
    for var in thisroc.data_vars:
        fig = plt.figure(figsize=(18,10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
        avalues = thisroc.sel(category=icat)[var].values
        cs = plt.contourf(thisroc[var].lon,thisroc[var].lat,avalues,levels=np.linspace(np.nanmin(avalues), np.nanmax(avalues), 20),cmap='YlGn', extend='min')
        cs.cmap.set_under('lightgray')
        cb = plt.colorbar(shrink=0.5)
        cb.ax.set_ylabel('Area under ROCSS curve',fontsize=12)
        plt.title(tit_line1 + f' {VARNAMES[var]}' + f' ({CATNAMES[icat]})\n' + tit_line2, loc='left')
        plt.tight_layout()  
        plt.show()

file_droptown=widgets.Dropdown(options=available_files,value="cmcc_35",description="Center")
cat_slider=widgets.IntSlider(value=3,min=0,max=2,step=1,description="cat")
fcmonth_slider=widgets.IntSlider(value=3,min=2,max=4,step=1,description="fcmonth")
aggr_droptown=widgets.Dropdown(options=["1m","3m"],value="1m",description="aggr")
widgets.interactive(plot_roc,file=file_droptown,aggr=aggr_droptown,fcmonth=fcmonth_slider,icat=cat_slider)

interactive(children=(Dropdown(description='Center', index=6, options=('ukmo_602', 'meteo_france_8', 'ecmwf_51…

## *E- BRIER SCORE (BS)*

In [7]:
BS=load_data("ukmo_602","1m","bs")
BS

<xarray.Dataset> Size: 316kB
Dimensions:        (lon: 91, lat: 48, forecastMonth: 3, category: 3)
Coordinates:
  * lon            (lon) float64 728B -30.0 -29.0 -28.0 -27.0 ... 58.0 59.0 60.0
  * lat            (lat) float64 384B 44.5 43.5 42.5 41.5 ... 0.5 -0.5 -1.5 -2.5
  * forecastMonth  (forecastMonth) int32 12B 2 3 4
  * category       (category) int32 12B 0 1 2
    number         int32 4B ...
    step           timedelta64[ns] 8B ...
    surface        float64 8B ...
    start_month    int32 4B ...
Data variables:
    t2m            (forecastMonth, category, lat, lon) float64 314kB ...

In [8]:
def plot_roc(file,aggr,fcmonth,icat):
    origin_labels = {'institution': f'{file}',
                    'SCORE':'BS'}
    
    tit_line1 = '{institution} : {SCORE} '.format(**origin_labels)
    tit_line2_base = f'Start month: {calendar.month_abbr[config["start_month"]].upper()}'
    
    if aggr=='1m':
        validmonth = config['start_month'] + (fcmonth-1)
        validmonth = validmonth if validmonth<=12 else validmonth-12
    #     tit_line2 = tit_line2_base + rf' - Valid month: $\bf{calendar.month_abbr[validmonth].upper()}$'
        tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
    elif aggr=='3m':
        validmonths = [vm if vm<=12 else vm-12 for vm in [config['start_month'] + (fcmonth-1) - shift for shift in range(3)]]
        validmonths = [calendar.month_abbr[vm][0] for vm in reversed(validmonths)]
    #     tit_line2 = tit_line2_base + rf' - Valid months: $\bf{"".join(validmonths)}$'
        tit_line2 = tit_line2_base + f' - Valid months: {"".join(validmonths)}'
    else:
        raise BaseException(f'Unexpected aggregation {aggr}')
    
    roc=load_data(file,aggr,"bs")
    thisroc = roc.sel(forecastMonth=fcmonth)
    for var in thisroc.data_vars:
        fig = plt.figure(figsize=(18,10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
        avalues = thisroc.sel(category=icat)[var].values
        cs = plt.contourf(thisroc[var].lon,thisroc[var].lat,avalues,levels=np.linspace(np.nanmin(avalues), np.nanmax(avalues), 20),cmap='YlGn', extend='min')
        cs.cmap.set_under('lightgray')
        cb = plt.colorbar(shrink=0.5)
        cb.ax.set_ylabel('BRIER SCORE',fontsize=12)
        plt.title(tit_line1 + f' {VARNAMES[var]}' + f' ({CATNAMES[icat]})\n' + tit_line2, loc='left')
        plt.tight_layout()  
        plt.show()

file_droptown=widgets.Dropdown(options=available_files,value="cmcc_35",description="Center")
cat_slider=widgets.IntSlider(value=3,min=0,max=2,step=1,description="cat")
fcmonth_slider=widgets.IntSlider(value=3,min=2,max=4,step=1,description="fcmonth")
aggr_droptown=widgets.Dropdown(options=["1m","3m"],value="1m",description="aggr")
widgets.interactive(plot_roc,file=file_droptown,aggr=aggr_droptown,fcmonth=fcmonth_slider,icat=cat_slider)

interactive(children=(Dropdown(description='Center', index=6, options=('ukmo_602', 'meteo_france_8', 'ecmwf_51…